In [51]:
!pip install rouge

In [15]:
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from rouge import Rouge
ROUGE = Rouge()

In [16]:
df = pd.read_csv("C:\\Users\\Tanvi Kandalla\\Desktop\\NLP Assigments\\combined.csv", lineterminator='\n')

In [18]:
df = df.drop("rouge-1 score\r", axis = 1)

In [19]:
df

,title,abstract,resolved abstract,summary
0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ...","In our study, two dairy compost heaps and one ...",and VRE in compost were
1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...,We describe geometric non-commutative formal g...,formal groups in terms of a geometric commutat...
2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...,In the numerical solution of partial different...,inefficient mesh quality metric with a more ef...
3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...,Qualitative case study is hardly a research te...,is typical of most qualitative research to emp...
4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...,Resumen. En la actualidad están apareciendo un...,la arquitectura de un motor de transformación ...
...,...,...,...,...
95,ExoMol molecular line lists-XXVI : spectra of ...,Line lists for the sulphur-containing molecule...,Line lists for the sulphur-containing molecule...,NS calculated line list includes around 2.8 mi...
96,Multiround Private Information Retrieval: Capa...,Private information retrieval (PIR) is the pro...,Private information retrieval (PIR) is the pro...,The capacity of PIR has recently been characte...
97,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,frequency domain surface integral equation pro...
98,Are the Determinants of Markup Size Industry-S...,The aim of this paper is to identify factors t...,The aim of this paper is to identify factors t...,affect the pricing policy in Slovenian manufac...


In [20]:
from collections import OrderedDict
import numpy as np


class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def sentence_segment(self, doc, lower):
        sentences = []
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sents = re.split(pattern, doc)
        sents = [sentence.split() for sentence in sents if any(char.isalpha() for char in sentence)]
        
        for sent in sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if lower is True:
                    selected_words.append(token.lower())
                else:
                    selected_words.append(token)
            sentences.append(selected_words)
        return sentences
        
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                window_size=4, lower=False):
        """Main function to analyze text"""
        
        # Filter sentences
        sentences = self.sentence_segment(text, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [1]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer

def summarize_text(dataframe, weights):
    # Filling in NaN values
    if dataframe['resolved abstract'].isnull().any():
        dataframe['resolved abstract'] = dataframe['resolved abstract'].fillna('This abstract does not exist')

    # Getting tf-idf tables
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(dataframe['resolved abstract'])
    feature_names = vectorizer.get_feature_names()

    # Generating best title for each abstract
    summaries = []
    scores = []
    for i in range(len(dataframe)):
        abstract = dataframe.loc[i, 'resolved abstract']
        title = dataframe.loc[i, 'title']
        tfidf_scores = tfidf_matrix[i].toarray().flatten()
        
        textRank = TextRank4Keyword()
        textRank.analyze(abstract)
        text_rank_scores = textRank.node_weight
        
        abstract = abstract.translate(str.maketrans('', '', string.punctuation))
        # Sentence Tokenization
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sentences = re.split(pattern, abstract)
        sentences = [sentence for sentence in sentences if any(char.isalpha() for char in sentence)]

        # N-gram scoring based on tf-idf values
        max_score = -1
        best_summary = ''
        s = {}
        for sentence in sentences:
            words = sentence.split(' ')
            ngrams_list = list(ngrams(words, len(title.split())))
            score = 0
            for ngram in ngrams_list:
                tfidf_score = sum(tfidf_scores[vectorizer.vocabulary_.get(word.lower(), -1)] for word in ngram)
                if ngram in list(text_rank_scores.keys()):
                    tr_score = (text_rank_scores[ngram])
                else:
                    tr_score = 0.0
                # Combine scores using weights
                score += weights['tfidf'] * tfidf_score + weights['textrank'] * tr_score

        
                wordlist = words
                stop_words = set(stopwords.words('english'))
                wordlist = [w for w in wordlist if not w in stop_words]

                tagged = nltk.pos_tag(wordlist)

                for i in tagged:
                    if i[1][0] == 'N' and i in wordlist:
                        s[i[0]] += weights['nouns']
        
        max_score = -1
        if (score > max_score):
                    max_score = score
                    summary = ' '.join(ngram)
        best_summary = summary
        summaries.append(best_summary)
        if title and best_summary:
            score = ROUGE.get_scores(title, best_summary)
            p = score[0]["rouge-1"]["p"]
            r = score[0]["rouge-1"]["r"]
            f1 = score[0]["rouge-1"]["f"]
            scores.append([p, r, f1])
        else:
            scores.append([0.0, 0.0, 0.0])

    dataframe['summary'] = summaries
    dataframe['rouge-1 score'] = scores

    return dataframe, s


In [22]:
import itertools

def grid_search_weights(dataframe, weight_ranges):
    best_weights = None
    best_score = -1

    # Generate all possible combinations of weights within the specified ranges
    weight_combinations = list(itertools.product(*weight_ranges.values()))

    for weights in weight_combinations:
        # Set the weights for each feature
        weight_dict = {feature: weight for feature, weight in zip(weight_ranges.keys(), weights)}

        # Summarize text using the current weights
        summaries, s = summarize_text(dataframe, weights=weight_dict)

        # Evaluate the performance using an appropriate evaluation metric
        score = evaluate_summaries(summaries,'rouge-1 score')

        # Keep track of the best weights and score
        if score > best_score:
            best_score = score
            best_weights = weight_dict

    return best_weights, best_score

In [23]:
df

,title,abstract,resolved abstract,summary
0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ...","In our study, two dairy compost heaps and one ...",and VRE in compost were
1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...,We describe geometric non-commutative formal g...,formal groups in terms of a geometric commutat...
2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...,In the numerical solution of partial different...,inefficient mesh quality metric with a more ef...
3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...,Qualitative case study is hardly a research te...,is typical of most qualitative research to emp...
4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...,Resumen. En la actualidad están apareciendo un...,la arquitectura de un motor de transformación ...
...,...,...,...,...
95,ExoMol molecular line lists-XXVI : spectra of ...,Line lists for the sulphur-containing molecule...,Line lists for the sulphur-containing molecule...,NS calculated line list includes around 2.8 mi...
96,Multiround Private Information Retrieval: Capa...,Private information retrieval (PIR) is the pro...,Private information retrieval (PIR) is the pro...,The capacity of PIR has recently been characte...
97,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,frequency domain surface integral equation pro...
98,Are the Determinants of Markup Size Industry-S...,The aim of this paper is to identify factors t...,The aim of this paper is to identify factors t...,affect the pricing policy in Slovenian manufac...


In [24]:
weight_ranges = {
    'tfidf': [0.2, 0.4, 0.6, 0.8, 1.0],
    'nouns': [0.1, 0.2, 0.3, 0.4, 0.5, 1.0],
    'textrank': [0.1, 0.2, 0.3, 0.4, 0.5, 1.0]
}


In [25]:
def evaluate_summaries(dataframe,col):
    p_sum = 0
    for score in dataframe[col]:
        p_sum += score[0]
    total = len(dataframe[col])
    avg = p_sum/total
    return avg

In [26]:
df['title']

0         A Microbiological Survey and Characterization
1     eb 2 00 6 Non-Commutative Formal Groups in Pos...
2     An Alternating Mesh Quality Metric Scheme for ...
3     Researching Distance Learning Experiences Usin...
4     Un Motor de Transformación de Modelos con Sopo...
                            ...                        
95    ExoMol molecular line lists-XXVI : spectra of ...
96    Multiround Private Information Retrieval: Capa...
97    Efficient Integral Equation Algorithms and The...
98    Are the Determinants of Markup Size Industry-S...
99    Distributed Recovery of Node Failures in Wirel...
Name: title, Length: 100, dtype: object

In [27]:
weights = grid_search_weights(df.head(10),weight_ranges)[0]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_27064\4060892020.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['summary'] = summaries
C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_27064\4060892020.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [28]:
weights

{'tfidf': 0.2, 'nouns': 0.1, 'textrank': 0.1}

In [29]:
output, s = summarize_text(df.head(50),weights)

C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_27064\4060892020.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['summary'] = summaries
C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_27064\4060892020.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['rouge-1 score'] = scores


In [30]:
output

,title,abstract,resolved abstract,summary,rouge-1 score
0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ...","In our study, two dairy compost heaps and one ...",,"[0.0, 0.0, 0.0]"
1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...,We describe geometric non-commutative formal g...,,"[0.0, 0.0, 0.0]"
2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...,In the numerical solution of partial different...,,"[0.0, 0.0, 0.0]"
3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...,Qualitative case study is hardly a research te...,,"[0.0, 0.0, 0.0]"
4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...,Resumen. En la actualidad están apareciendo un...,,"[0.0, 0.0, 0.0]"
5,Phonetics in Functional Discourse Grammar,The theory of Functional Discourse Grammar ass...,The theory of Functional Discourse Grammar ass...,,"[0.0, 0.0, 0.0]"
6,ASLA: reverse engineering approach for softwar...,Software maintenance and reuse require identif...,Software maintenance and reuse require identif...,,"[0.0, 0.0, 0.0]"
7,Implementing interference cancellation to incr...,This article provides the principles and pract...,This article provides the principles and pract...,,"[0.0, 0.0, 0.0]"
8,Smart detectors for Monte Carlo radiative tran...,Many optimization techniques have been invente...,Many optimization techniques have been invente...,,"[0.0, 0.0, 0.0]"
9,The Use of Reversed in Situ Cartilage Flaps Du...,Introduction: Problems faced with cartilage gr...,Introduction: Problems faced with cartilage gr...,,"[0.0, 0.0, 0.0]"


In [494]:
(output['rouge-1 score\r'][1])

'[0.4, 0.36363636363636365, 0.380952380952381]\r'

In [498]:
import ast

for i in range(0,len(output['rouge-1 score\r'])):
    stringg = output['rouge-1 score\r'][i]

    # Remove '\r' and convert to list
    clean_string = stringg.strip()
    result_list = ast.literal_eval(clean_string)
    
    output['rouge-1 score\r'][i] = result_list

C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_27560\173722158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['rouge-1 score\r'][i] = result_list


In [486]:
p_sum = 0
for score in output['rouge-1 score\r']:
        p_sum += score[0]
total = len(output['rouge-1 score\r'])

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [499]:
evaluate_summaries(output,'rouge-1 score\r')

0.40371364746364746

In [88]:
# Batch processing the dataframe 1000 rows at a time
output = []

i = 1
count = 0
while count < 500000/1000:
    if(i==1):
        output.append(summarize_text(df[:1001],weights))
    else:
        output.append(summarize_text(df[i:i+1000].reset_index(),weights))
    i += 1000
    count += 1

/tmp/ipykernel_708/2312363059.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['summary'] = summaries
/tmp/ipykernel_708/2312363059.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['rouge-1 score'] = scores


In [89]:
# Adding together all the dataframes
concatenated_df = pd.DataFrame()

for df_slice in output:
    concatenated_df = pd.concat([concatenated_df, df_slice])

concatenated_df = concatenated_df.reset_index(drop=True)
concatenated_df = concatenated_df.drop('index', axis=1)

In [181]:
evaluate_summaries(output)

0.03413092463092464

In [90]:
print(df['abstract'][4])

With the tremendous amount of information that becomes available on the Web on a daily basis, the ability to quickly develop information agents has become a crucial problem. A vital component of any Web-based information agent is a set of wrappers that can extract the relevant data from semistructured information sources. Our novel approach to wrapper induction is based on the idea of hierarchical information extraction, which turns the hard problem of extracting data from an arbitrarily complex document into a series of easier extraction tasks. We introduce an inductive algorithm, stalker, that generates high accuracy extraction rules based on user-labeled training examples. Labeling the training data represents the major bottleneck in using wrapper induction techniques, and our experimental results show that stalker does signi cantly better then other approaches; on one hand, stalker requires up to two orders of magnitude fewer examples than other algorithms, while on the other hand 

In [91]:
df = concatenated_df

In [92]:
print(df['summary'][4])

to wrapper induction is based on


In [93]:
print(df['title'][4])

A Hierarchical Approach to Wrapper Induction


In [288]:
output

,title,abstract,resolved abstract,summary,rouge-1 score\r,rouge-1 score
0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ...","In our study, two dairy compost heaps and one ...","4.6, 4.07, 3.97 After In our study, 30 1.6 th...","[0.2, 0.2, 0.20000000000000004]\r","[0.0, 0.0, 0.0]"
1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...,We describe geometric non-commutative formal g...,Although be us characterizing made precise be...,"[0.4, 0.36363636363636365, 0.380952380952381]\r","[0.0, 0.0, 0.0]"
2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...,In the numerical solution of partial different...,"When Typical More many applications, this we ...","[0.5454545454545454, 0.5454545454545454, 0.545...","[0.0, 0.0, 0.0]"
3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...,Qualitative case study is hardly a research te...,"Specifically, Some The are made considers beg...","[0.3125, 0.35714285714285715, 0.33333333333333...","[0.0, 0.0, 0.0]"
4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...,Resumen. En la actualidad están apareciendo un...,"No obstante, actualidad no Resumen. están apa...","[0.6428571428571429, 0.6428571428571429, 0.642...","[0.0, 0.0, 0.0]"
...,...,...,...,...,...,...
95,ExoMol molecular line lists-XXVI : spectra of ...,Line lists for the sulphur-containing molecule...,Line lists for the sulphur-containing molecule...,Each simulated ab using potential dipole init...,"[0.36363636363636365, 0.4, 0.380952380952381]\r","[0.0, 0.0, 0.0]"
96,Multiround Private Information Retrieval: Capa...,Private information retrieval (PIR) is the pro...,Private information retrieval (PIR) is the pro...,Here In provide every consider We through we ...,"[0.125, 0.125, 0.125]\r","[0.0, 0.0, 0.0]"
97,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,Overlapping sub-domain The Efficient required...,"[0.3, 0.3, 0.3]\r","[0.1, 0.09090909090909091, 0.09523809024943337]"
98,Are the Determinants of Markup Size Industry-S...,The aim of this paper is to identify factors t...,The aim of this paper is to identify factors t...,"We main find The each aim this Accordingly, W...","[0.5384615384615384, 0.5, 0.5185185185185186]\r","[0.08333333333333333, 0.09090909090909091, 0.0..."


In [97]:
df['rouge-1 p score'] = 0.0
df['rouge-1 r score'] = 0.0
df['rouge-1 f1 score'] = 0.0

for i in range(0,len(df['rouge-1 score'])):
    df['rouge-1 p score'][i] = df['rouge-1 score'][i][0]
    df['rouge-1 r score'][i] = df['rouge-1 score'][i][1]
    df['rouge-1 f1 score'][i] = df['rouge-1 score'][i][2]

/tmp/ipykernel_708/3305449461.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rouge-1 p score'][i] = df['rouge-1 score'][i][0]
/tmp/ipykernel_708/3305449461.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rouge-1 r score'][i] = df['rouge-1 score'][i][1]
/tmp/ipykernel_708/3305449461.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rouge-1 f1 score'][i] = df['rouge-1 score'][i][2]


In [99]:
df = df.drop('rouge-1 score',axis = 1)

In [102]:
p_sum = r_sum = f1_sum = 0
for i in range(0,len(df['rouge-1 p score'])):
    p_sum += df['rouge-1 p score'][i]
    r_sum += df['rouge-1 r score'][i]
    f1_sum += df['rouge-1 f1 score'][i]
total = len(df['rouge-1 p score'] )
avg = [p_sum/total, r_sum/total, f1_sum/total]
print(avg)

[0.23045599899994798, 0.2394645544732437, 0.23437250793925912]


In [103]:
df

,title,abstract,summary,rouge-1 p score,rouge-1 r score,rouge-1 f1 score
0,A family of tridiagonal pairs related to the q...,"A type of tridiagonal pair is considered, said...",is shown that these tridiagonal pairs induce t...,0.388889,0.368421,0.378378
1,Reduction of secondary variables on soft-sensi...,A novel method based on partial least-squares ...,proposed on select the most suitable secondary...,0.250000,0.230769,0.240000
2,Design of 2MW/10kV cascaded power conversion s...,Cascaded power conversion system was used in b...,cascaded power conversion system can reduce power,0.571429,0.666667,0.615385
3,Modeling charge polarization voltage for large...,Purpose: Polarization voltage of the lithium-i...,"of currents, initial SOC of the battery on cha...",0.272727,0.300000,0.285714
4,A Hierarchical Approach to Wrapper Induction,With the tremendous amount of information that...,to wrapper induction is based on,0.166667,0.166667,0.166667
...,...,...,...,...,...,...
499995,Narrowband MIMO Channel Modeling for LOS Indoo...,"Herein, a line-of-sight (LOS) multiple input m...",model the whole narrowband LOS indoor MIMO cha...,0.250000,0.250000,0.250000
499996,Review on Methods of Self-Balancing of Two Inl...,Self-balancing of two inline wheels has been a...,of gyroscopic stabilization in a two inline wh...,0.125000,0.111111,0.117647
499997,Is it Just that People with Higher Incomes Can...,Empfohlene Zitierung / Suggested Citation: Lin...,Empfohlene Zitierung / Suggested Citation: Lin...,0.666667,0.666667,0.666667
499998,A Survey on Multi-View Clustering,With advances in information acquisition techn...,a survey to summarize and,0.000000,0.000000,0.000000


In [104]:
df.to_csv("Doc2_summaries_new.csv",index=None)

In [105]:
df2 = pd.read_csv("SummarizationCSTitleAbstract03(1).csv",lineterminator='\n')

In [80]:
# Batch processing the dataframe 1000 rows at a time
output2 = []

i = 1
count = 0
while count < 500000/1000:
    if(i==1):
        output2.append(summarize_text(df2[:1001],weights))
    else:
        output2.append(summarize_text(df2[i:i+1000].reset_index(),weights))
    i += 1000
    count += 1

/tmp/ipykernel_708/2312363059.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['abstract'] = dataframe['abstract'].fillna('This abstract does not exist')
/tmp/ipykernel_708/2312363059.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['summary'] = summaries
/tmp/ipykernel_708/2312363059.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [81]:
# Adding together all the dataframes
concatenated_df2 = pd.DataFrame()

for df_slice in output2:
    concatenated_df2 = pd.concat([concatenated_df2, df_slice])

concatenated_df2 = concatenated_df2.reset_index(drop=True)
concatenated_df2 = concatenated_df2.drop('index', axis=1)

In [106]:
df2 = concatenated_df2

In [108]:
df2['rouge-1 p score'] = 0.0
df2['rouge-1 r score'] = 0.0
df2['rouge-1 f1 score'] = 0.0

for i in range(0,len(df2['rouge-1 score'])):
    df2['rouge-1 p score'][i] = df2['rouge-1 score'][i][0]
    df2['rouge-1 r score'][i] = df2['rouge-1 score'][i][1]
    df2['rouge-1 f1 score'][i] = df2['rouge-1 score'][i][2]

/tmp/ipykernel_708/2714953272.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['rouge-1 p score'][i] = df2['rouge-1 score'][i][0]
/tmp/ipykernel_708/2714953272.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['rouge-1 r score'][i] = df2['rouge-1 score'][i][1]
/tmp/ipykernel_708/2714953272.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['rouge-1 f1 score'][i] = df2['rouge-1 score'][i][2]


In [109]:
df2 = df2.drop('rouge-1 score',axis = 1)

In [110]:
p_sum = r_sum = f1_sum = 0
for i in range(0,len(df2['rouge-1 p score'])):
    p_sum += df2['rouge-1 p score'][i]
    r_sum += df2['rouge-1 r score'][i]
    f1_sum += df2['rouge-1 f1 score'][i]
total = len(df2['rouge-1 p score'] )
avg = [p_sum/total, r_sum/total, f1_sum/total]
print(avg)

[0.22978363877593297, 0.23897536173364844, 0.23378030899481284]


In [111]:
df2

,Unnamed: 0,title,abstract,summary,rouge-1 p score,rouge-1 r score,rouge-1 f1 score
0,0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ...",and 1.6 logs at the,0.200000,0.166667,0.181818
1,1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...,formal groups in terms of a geometric commutat...,0.100000,0.111111,0.105263
2,2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...,inefficient mesh quality metric with a more ef...,0.000000,0.000000,0.000000
3,3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...,Qualitative case study is hardly a research te...,0.133333,0.142857,0.137931
4,4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...,de un motor de transformación de modelos que p...,0.166667,0.181818,0.173913
...,...,...,...,...,...,...,...
499995,499995,Investigation of Teachers ' Awareness of Flipp...,Although Flipped Classroom(FC) which has recen...,and pre-service teachers awareness of FC in or...,0.250000,0.230769,0.240000
499996,499996,A Practical Parallel Algorithm for Cycle Detec...,Graph theoretic techniques are used in variety...,"present a new, parallel algorithm for detectin...",0.222222,0.222222,0.222222
499997,499997,A Transition to Fast Flows and Its Effects on ...,A shocklike transition at 2003.37 from quasi-p...,The changes in the magnetic field associated w...,0.047619,0.055556,0.051282
499998,499998,Patterned turbulence and relaminarization in M...,We present results of a numerical analysis of ...,numerical analysis of relaminarization process...,0.666667,0.600000,0.631579


In [112]:
df2.to_csv("Doc1_summaries_new.csv",index=None)